In [78]:
import pandas as pd
import json
import re
import sys
import os
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
import tempfile

sys.path.append(os.path.abspath(".."))

def dummy_npwarn_decorator_factory():
  # https://stackoverflow.com/questions/77064579/module-numpy-has-no-attribute-no-nep50-warning SACADO DE AQUI
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)
# import nltk
# nltk.download('punkt')  # Asegúrate de esto

# from nltk.tokenize import PunktSentenceTokenizer

from AUDIO.scripts.AudioEmotionAnalzer import AudioEmotionAnalysis
from TEXT.POO.scripts.TextEmotionAnalizer import TextEmotionClassifier

In [79]:
# df = pd.read_csv('/home/aacastro/Alejandro/ACA_MultichanelAI_2025/src/TEXT/POO/annotated/AAPL/2018/Q3.csv')
# df.head()

# tokenizer = PunktSentenceTokenizer()

# df['sentences'] = df['text'].apply(lambda x: tokenizer.tokenize(x) if pd.notnull(x) else [])

In [80]:
df_csv = pd.read_csv('/home/aacastro/Alejandro/ACA_MultichanelAI_2025/src/TEXT/POO/annotated/AAPL/2018/Q3.csv')
with open('/home/aacastro/mchai/companies/AAPL/2018/Q3/LEVEL_3.json', 'r') as f:
    data_json = json.load(f)

# Transformación de intervenciones completas a frases con timestamps

En este notebook trabajamos con dos fuentes de datos complementarias:

- Un archivo CSV que contiene intervenciones completas, además de metadatos asociados como el nombre y título del hablante, la sección de la conferencia, la clasificación de la intervención (por ejemplo, presentación, pregunta o respuesta), el nivel de confianza y el par pregunta-respuesta correspondiente.
- Un archivo JSON que almacena la transcripción palabra por palabra de esas intervenciones, incluyendo para cada palabra su tiempo de inicio en segundos. Además, se especifica qué hablante pronunció cada conjunto de palabras.

El objetivo del procesamiento es generar un nuevo conjunto de datos en el que cada fila represente una **única frase**, con el tiempo de inicio y final estimado de dicha frase. De esta forma, se facilita el análisis temporal, lingüístico o multimodal (por ejemplo, sincronización con audio o vídeo) a nivel de frase sin perder el contexto original de la intervención.

---

## Extracción de frases desde el archivo JSON

El archivo JSON proporciona una segmentación precisa a nivel de palabra, junto con los timestamps de inicio. Para obtener frases completas, se agrupan palabras hasta detectar una marca de puntuación que indique final de frase (como el punto, el signo de exclamación o de interrogación).

Para cada frase detectada se almacena el texto completo de la frase, el nombre del hablante que la pronunció, el tiempo de inicio de la primera palabra y el tiempo de final de la última palabra. El resultado es una lista de frases con su marca temporal correspondiente.

---

## División de las intervenciones del CSV en frases

Las intervenciones del archivo CSV suelen consistir en párrafos largos con múltiples frases. Para poder alinear esta información con los timestamps extraídos del JSON, es necesario dividir cada intervención en frases individuales. Esta segmentación se realiza buscando los mismos signos de puntuación que se usaron en el paso anterior.

Una vez dividida la intervención en frases, se replica toda la información contextual de la intervención original (nombre, título, sección, clasificación, etc.) para cada nueva frase creada. Así se conserva toda la riqueza de información del CSV en el nuevo formato.

---

## Alineación y asignación de tiempos

Una vez tenemos dos listas de frases —una con la información y metadatos del CSV, y otra con los tiempos del JSON— se asume que ambas listas están en el mismo orden y que cada frase del CSV coincide, en texto y posición, con una frase del JSON.

Si el número de frases no coincide exactamente, se trunca la lista más larga para evitar desalineaciones. Después, se asigna el tiempo de inicio y final de cada frase del JSON a la correspondiente frase del CSV.

---

## Resultado final

El resultado es un DataFrame enriquecido en el que cada fila representa una frase individual. Cada una de estas frases incluye:

- Todos los metadatos de la intervención original (como si no se hubiese segmentado).
- El texto de la frase concreta.
- El `start_time` y `end_time` de la frase, calculados desde la transcripción palabra por palabra del JSON.

Este nuevo formato permite trabajar directamente a nivel de frase con total contexto y soporte temporal.

In [81]:
# 2. Extraer frases del JSON con start y end time
frases_json = []

for speaker in data_json["speakers"]:
    words = speaker["words"]
    times = speaker["start_times"]
    speaker_name = speaker["speaker_info"]["name"]

    frase = ""
    tiempos = []

    for palabra, tiempo in zip(words, times):
        frase += palabra + " "
        tiempos.append(tiempo)

        if re.match(r".*[\.\!\?]$", palabra):  # fin de frase
            frases_json.append({
                "speaker": speaker_name,
                "text": frase.strip(),
                "start_time": tiempos[0],
                "end_time": tiempos[-1]
            })
            frase = ""
            tiempos = []

df_json = pd.DataFrame(frases_json)


# 3. Dividir cada intervención del CSV en frases y replicar metadatos
frases_expandidas = []

for _, row in df_csv.iterrows():
    frases = re.split(r'(?<=[\.\!\?])\s+', row["text"])  # frases del texto
    for frase in frases:
        if frase.strip():
            nueva_fila = row.to_dict()
            nueva_fila["text"] = frase.strip()  # frase individual
            frases_expandidas.append(nueva_fila)

df_expandido = pd.DataFrame(frases_expandidas)


# 4. Asignar start_time y end_time desde el JSON (asumiendo orden)
if len(df_expandido) != len(df_json):
    print(f"⚠️ Atención: Nº frases CSV ({len(df_expandido)}) != Nº frases JSON ({len(df_json)})")

# Añadir tiempos si hay correspondencia
min_len = min(len(df_expandido), len(df_json))
df_expandido = df_expandido.iloc[:min_len].copy()
df_json = df_json.iloc[:min_len].copy()

df_expandido["start_time"] = df_json["start_time"].values
df_expandido["end_time"] = df_json["end_time"].values

⚠️ Atención: Nº frases CSV (429) != Nº frases JSON (428)


In [82]:
# df_expandido.to_csv('sentences.csv')
df_expandido.head(20)

clasificaciones_deseadas = ['Presentation', 'Question', 'Answer']
df_filtrado = df_expandido[df_expandido['classification'].isin(clasificaciones_deseadas)]

In [83]:
for _, row in df_filtrado.head(5).iterrows():
    print(f"[{row['classification']}] {row['text']} ({row['start_time']}s) ({row['end_time']}s)")

[Presentation] Please stand by. (0.765s) (1.385s)
[Presentation] We're about to begin. (1.626s) (2.206s)
[Presentation] Good day, and welcome to the Apple Incorporated third quarter fiscal year 2018 earnings conference call. (3.467s) (8.13s)
[Presentation] Today's call is being recorded. (8.931s) (9.931s)
[Presentation] At this time, for opening remarks and introductions, I'd like to turn the call over to Nancy Paxton, Senior Director of Investor Relations. (10.832s) (16.276s)


### Orden de las emociones en los embeddings: ['happy', 'neutral', 'surprise', 'disgust', 'anger', 'sadness', 'fear']

## Text

In [84]:
frase = 'Good day, and welcome to the Apple Incorporated third quarter fiscal year 2018 earnings conference call.'

clf_text = TextEmotionClassifier()

text_embeddings = clf_text.get_embeddings(frase)
text_embeddings

Device set to use cuda:0


tensor([ 4.1922,  2.3097,  0.8478, -1.3362, -1.8390, -1.9948, -2.1797])

## Audio

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import tempfile

def cortar_audio_temporal(input_path, start_time: int, end_time: int):
    """
    Recorta un archivo MP3 entre un rango de tiempo específico y devuelve un archivo temporal WAV.

    Args:
        input_file (str): Ruta del archivo MP3 de entrada.
        start_time (int): Tiempo inicial en segundos.
        end_time (int): Tiempo final en segundos.

    Returns:
        tempfile.NamedTemporaryFile: Archivo temporal WAV listo para usar.
    """
    try:
        # Cargar el audio
        audio = AudioSegment.from_mp3(input_path)

        # Convertir tiempos a milisegundos
        # start_ms = start_time * 1000
        # end_ms = end_time * 1000
        start_ms = int(start_time * 1000)
        end_ms = int((end_time + 0.25) * 1000)

        # Cortar el segmento
        segmento = audio[start_ms:end_ms]
        display(Audio(segmento.export(format="mp3").read(), rate=44100))

        # Crear archivo temporal WAV
        temp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        segmento.export(temp_wav.name, format="wav")

        return temp_wav

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

In [89]:
audio_path = "/home/aacastro/mchai/RAVDESS/audios/Actor_01/03-01-05-02-01-01-01.wav"

clf_audio = AudioEmotionAnalysis()

audio_embeddings = clf_audio.get_embeddings(audio_path)
audio_embeddings

2025-03-30 15:31:27,778 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: /home/aacastro/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
ckpt: /home/aacastro/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init param, map: modal

rtf_avg: 0.016: 100%|██████████| 1/1 [00:00<00:00, 13.07it/s]                                                                                      


tensor([-1.6415, -2.8577, -0.3029, -5.5051, 18.1174, -2.8723, -4.9379])

## Integration

In [ ]:
input_path = "/home/aacastro/mchai/companies/AAPL/2018/Q3/audio.mp3"

# Itera sobre los primeros 5 segmentos
for _, row in df_filtrado.head(5).iterrows():
    print("="*70)

    temp_file = cortar_audio_temporal(input_path, row['start_time'], row['end_time'])
    
    # Mostrar información y reproducir
    print(f"[{row['classification']}] {row['text']} ({row['start_time']}s - {row['end_time']}s)")
    
    # display(Audio(segmento.export(format="mp3").read(), rate=44100))

    # Obtener embeddings y printearlos
    audio_embeddings = clf_audio.get_embeddings(temp_file.name)
    text_embeddings = clf_text.get_embeddings(row['text'])
    print("AUDIO EMBEDDINGS: ", audio_embeddings)
    print("TEXT EMBEDDINGS: ", text_embeddings)
    print('')
    os.remove(temp_file.name)

[Presentation] Please stand by. (0.765s - 1.385s)


rtf_avg: 0.083: 100%|██████████| 1/1 [00:00<00:00, 13.64it/s]                                                                                      


AUDIO EMBEDDINGS:  tensor([ 1.2100,  4.4502, -1.4657, -0.6059, -0.8751, -0.7740, -1.9395])
TEXT EMBEDDINGS:  tensor([ 2.4287,  1.2394,  0.6221,  0.4580, -0.1798, -1.7292, -2.8392])

[Presentation] We're about to begin. (1.626s - 2.206s)


rtf_avg: 0.029: 100%|██████████| 1/1 [00:00<00:00, 39.00it/s]                                                                                      


AUDIO EMBEDDINGS:  tensor([ 1.0763, 11.8506, -5.1948, -1.5616, -4.2629, -0.5536, -1.3540])
TEXT EMBEDDINGS:  tensor([ 2.2893,  1.6928,  0.3557, -0.4992, -0.6751, -1.5691, -1.5944])

[Presentation] Good day, and welcome to the Apple Incorporated third quarter fiscal year 2018 earnings conference call. (3.467s - 8.13s)


...

AUDIO EMBEDDINGS:  tensor([21.5532, -5.8494, -4.5376, -0.2820, -3.1161, -4.6766, -3.0915])
TEXT EMBEDDINGS:  tensor([ 4.1922,  2.3097,  0.8478, -1.3362, -1.8390, -1.9948, -2.1797])

[Presentation] Today's call is being recorded. (8.931s - 9.931s)


rtf_avg: 0.021: 100%|██████████| 1/1 [00:00<00:00, 36.49it/s]                                                                                      


AUDIO EMBEDDINGS:  tensor([ 1.4785,  4.9817, -3.1188, -0.4892, -2.6206, -0.7518,  0.5201])
TEXT EMBEDDINGS:  tensor([ 3.8024,  0.7416, -0.5315, -0.7789, -0.8985, -1.0014, -1.3337])

[Presentation] At this time, for opening remarks and introductions, I'd like to turn the call over to Nancy Paxton, Senior Director of Investor Relations. (10.832s - 16.276s)


rtf_avg: 0.008: 100%|██████████| 1/1 [00:00<00:00, 21.01it/s]                                                                                      

AUDIO EMBEDDINGS:  tensor([ 4.7937,  9.8550, -5.7281, -2.2478, -1.8238, -2.0199, -2.8292])
TEXT EMBEDDINGS:  tensor([ 3.7052,  0.4916, -0.3299, -0.4649, -0.7707, -1.0359, -1.5954])



In [95]:
input_path = "/home/aacastro/mchai/companies/AAPL/2018/Q3/audio.mp3"

# Itera sobre los primeros 5 segmentos
for i, (_, row) in enumerate(df_filtrado.head(5).iterrows(), start=1):
    print("=" * 70)
    print(f"🎧 Segmento {i}")
    print("-" * 70)

    temp_file = cortar_audio_temporal(input_path, row['start_time'], row['end_time'])

    # Mostrar información textual
    print(f"📌 Clasificación: {row['classification']}")
    print(f"🗣️  Texto: {row['text']}")
    print(f"⏱️  Intervalo de tiempo: {row['start_time']}s - {row['end_time']}s")

    # Obtener embeddings
    audio_embeddings = clf_audio.get_embeddings(temp_file.name)
    text_embeddings = clf_text.get_embeddings(row['text'])

    # Mostrar embeddings
    print("\n🔊 Embeddings de AUDIO:")
    print(audio_embeddings)
    print("\n📝 Embeddings de TEXTO:")
    print(text_embeddings)
    
    print("\n")  # Espacio entre segmentos
    os.remove(temp_file.name)

🎧 Segmento 1
----------------------------------------------------------------------
📌 Clasificación: Presentation
🗣️  Texto: Please stand by.
⏱️  Intervalo de tiempo: 0.765s - 1.385s


rtf_avg: 0.025: 100%|██████████| 1/1 [00:00<00:00, 40.69it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 1.2100,  4.4502, -1.4657, -0.6059, -0.8751, -0.7740, -1.9395])

📝 Embeddings de TEXTO:
tensor([ 2.4287,  1.2394,  0.6221,  0.4580, -0.1798, -1.7292, -2.8392])


🎧 Segmento 2
----------------------------------------------------------------------


📌 Clasificación: Presentation
🗣️  Texto: We're about to begin.
⏱️  Intervalo de tiempo: 1.626s - 2.206s


rtf_avg: 0.031: 100%|██████████| 1/1 [00:00<00:00, 37.00it/s]                                                                                      

...

rtf_avg: 0.036: 100%|██████████| 1/1 [00:00<00:00, 21.15it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 1.4785,  4.9817, -3.1188, -0.4892, -2.6206, -0.7518,  0.5201])

📝 Embeddings de TEXTO:
tensor([ 3.8024,  0.7416, -0.5315, -0.7789, -0.8985, -1.0014, -1.3337])


🎧 Segmento 5
----------------------------------------------------------------------


📌 Clasificación: Presentation
🗣️  Texto: At this time, for opening remarks and introductions, I'd like to turn the call over to Nancy Paxton, Senior Director of Investor Relations.
⏱️  Intervalo de tiempo: 10.832s - 16.276s


rtf_avg: 0.012: 100%|██████████| 1/1 [00:00<00:00, 13.84it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 4.7937,  9.8550, -5.7281, -2.2478, -1.8238, -2.0199, -2.8292])

📝 Embeddings de TEXTO:
tensor([ 3.7052,  0.4916, -0.3299, -0.4649, -0.7707, -1.0359, -1.5954])


